### Unidad III. Agrupamiento y clasificación. 

## El análisis discriminante de dos poblaciones como método de comparación Multivariada y técnicas de clasificación. Ideas generales de Fischer.

- El análisis discriminante de dos poblaciones por técnicas de regresión. 
- Generalización del problema de clasificación a más de dos poblaciones.
- Otras técnicas paramétricas.
- Alternativas no paramétricas del análisis discriminante. Modelos Log-lineales.